# Utility Functions

In [9]:
%load_ext literary.notebook
import ast

The literary.notebook extension is already loaded. To reload it, use:
  %reload_ext literary.notebook


In [8]:
import sys
from typing_extensions import Literal

## Quoting

In rendering the docstring for the generated Python module source, it is necessary to escape triple quoted strings. Here we implement such a function:

In [3]:
def escape_triple_quotes(string: str, single_quote: Literal["'", '"'] = '"') -> str:
    """Escape triple quotes inside a string

    :param string: string to escape
    :param single_quote: single-quote character
    :return: escaped string
    """
    assert len(single_quote) == 1
    quote = single_quote * 3
    escaped_single_quote = rf"\{single_quote}"
    escaped_quote = escaped_single_quote * 3
    return string.replace(quote, escaped_quote)

Let's ensure that this function behaves as expected for the `"` quote

In [4]:
assert (
    escape_triple_quotes('''"""Hello, I'm a triple quoted string"""''')
    == r'''\"\"\"Hello, I'm a triple quoted string\"\"\"'''
)

and for the `'` quote

In [5]:
assert (
    escape_triple_quotes("""'''Hello, I'm a triple quoted string'''""", "'")
    == r"\'\'\'Hello, I'm a triple quoted string\'\'\'"
)

## AST Roundtrip

It's useful to be able to round-trip Python source via an AST transformer. Python versions older than 3.9.0 do not implement such a feature, requiring a third-party library.

In [6]:
# Backwards compatibility
if sys.version_info < (3, 9, 0):
    import astunparse
    import astunparse.unparser

    class ASTUnparser(astunparse.unparser.Unparser):
        """AST unparser with additional preference for triple-quoted multi-line strings"""

        def _Constant(self, tree):
            if isinstance(tree.value, str) and "\n" in tree.value:
                self.write(f'"""{escape_triple_quotes(tree.value)}"""')
                return

            super()._Constant(tree)

    # Monkey patch to ensure correctness
    astunparse.Unparser = ASTUnparser
    astunparse.unparser.Unparser = ASTUnparser

    from astunparse import unparse as unparse_ast
else:
    from ast import unparse as unparse_ast

Let's generate a test AST node

In [10]:
node = ast.parse("x = y = f'hello {__name__}'")

We won't assert that the result is equal, because the `unparse_ast` method is allowed to apply optimisations to the generated source code. Let's look at the result visually, though:

In [11]:
unparse_ast(node)

"x = y = f'hello {__name__}'"